# RL Trading Model Training & Evaluation

This notebook demonstrates how to train a reinforcement learning (RL) trading model using your full feature pipeline and visualize the results. It loads historical price data, extracts features, trains a PPO agent, and evaluates performance.

In [1]:
# ================================================
# 🔧 SETUP - Add src to Python Path
# ================================================

import sys
import os

# Add src directory to Python path so 'core' module can be found
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in path: {src_path}")

# Verify
print(f"📂 Working directory: {project_root}")
print(f"🔍 Python will search for modules in: {src_path}")
print("=" * 50)

✅ Added to Python path: d:\Dev\trading-bot\src
📂 Working directory: d:\Dev\trading-bot
🔍 Python will search for modules in: d:\Dev\trading-bot\src


In [2]:
# Section 6: Complete Model Test Report with Updated Charts
print("📋 Generating Complete Test Report with Updated Interactive Charts...")
print("=" * 70)

# Import our clean testing solution
from src.reporting.model_test_report import quick_test_model

# Run model test - charts will display directly in notebook
print("🚀 Running model test with improved chart design...")

quick_test_model(
    model_dir='models/rl_optimized',
    symbol='BTCUSDT', 
    test_candles=500  # Use 1500+ for better trade visualizationRecalculating higher timeframe
)

📋 Generating Complete Test Report with Updated Interactive Charts...
🚀 Running model test with improved chart design...
🔍 Testing model: models/rl_optimized
📊 Symbol: BTCUSDT, Candles: 500
📊 Loading data...
📥 Loading data for BTCUSDT...
🔧 Converting levels cache index to DatetimeIndex...
✅ Loaded levels cache: data\levels_cache\BTCUSDT-15m-levels.parquet
📊 Shape: 101,000 rows × 9 columns
🤖 Loading model and generating predictions...
✅ GPU Available: NVIDIA GeForce RTX 3080 (10.0GB)
🖥️ RL Training Device: cuda
🎯 Generating predictions for 500 candles...
📥 Loaded normalizer from: models/rl_optimized\normalizer.pkl
📥 Loaded normalizer from models/rl_optimized\normalizer.pkl
⚡ Pre-normalizing feature data...
✅ Pre-normalized 31 features
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
✅ Model loaded on cuda
✅ Test completed! Shape: (164, 13)
📊 Columns: ['step', 'signal', 'position_size', 'price', 'entry_price', 'position_shares', 'cash', 'cash_used', 'unrealized

,step,signal,position_size,price,entry_price,position_shares,cash,cash_used,unrealized_pnl,equity,step_pnl,traded,total_trades
datetime,,,,,,,,,,,,,
2025-10-12 02:15:00,336,-1,0.0,110447.44,110447.44,-0.000,100000.00000,0.00,0.0,100000.00,0.00000,True,1
2025-10-12 02:30:00,337,0,0.0,110243.85,0.00,0.000,100000.00000,0.00,0.0,100000.00,0.00000,True,2
2025-10-12 02:45:00,338,0,0.0,110119.02,0.00,0.000,100000.00000,0.00,0.0,100000.00,0.00000,True,3
2025-10-12 03:00:00,339,0,0.0,110708.30,0.00,0.000,100000.00000,0.00,0.0,100000.00,0.00000,True,4
2025-10-12 03:15:00,340,0,0.0,110960.34,0.00,0.000,100000.00000,0.00,0.0,100000.00,0.00000,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-13 18:00:00,495,-1,0.0,114751.33,114751.33,-0.000,100075.11990,0.00,0.0,100075.12,0.00000,True,162
2025-10-13 18:15:00,496,-1,0.0,115258.40,115258.40,-0.000,100075.11990,0.00,0.0,100075.12,0.00000,True,163
2025-10-13 18:30:00,497,-1,1.0,115205.07,115205.07,-0.868,100075.11990,99998.00,0.0,100075.12,0.00000,True,164
